In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn.model_selection import train_test_split

In [ ]:
# load the mystery data
with open("sample_data.pkl", "rb") as f:
    X, y = pickle.load(f)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5), sharey=True)
axes[0].scatter(X[:,0], y, alpha=0.3)
axes[1].scatter(X[:,1], y, alpha=0.3)

axes[0].set_xlabel("Feature $x_0$")
axes[1].set_xlabel("Feature $x_1$")
axes[0].set_ylabel("Predicted $y$")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to("cpu")
print(model)

In [ ]:
# Define loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
history = np.zeros((epochs,))

for epoch in range(epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    history[epoch] = loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {history[epoch]:.4f}")

print("Training complete!")


In [ ]:
plt.plot(history)
plt.xlabel("Epoch")
plt.title("Training Loss")

In [ ]:
with torch.no_grad():
    y_pred = model(X_test_tensor)

plt.scatter(y_test, y_pred)

In [ ]:
plt.plot(np.log(X_test[:,1]/X_test[:,0]), y_test, '.')